# Example Calculation
## Client: INTERNAL
---
## Project: develop hydraulic tools
## Calc: CALC-002, hydraulic sample calculations
---
## Authentication
> Stamp, Permit
---
## Revision History
|Revision | Date | Description | By | Reviewer|
| :-------| :----|:------------|:---|:--------|
|       | | DRAFT | KCD |  |
|         |          |            |     |    |

---

## Introduction

In [56]:
# dependancies
using Pkg
#Pkg.add("DataFrames")
#Pkg.add("GridInterpolations")
#Pkg.add("Test")
using DataFrames
using GridInterpolations
#using Test
using CSV
using ExcelFiles


In [57]:
# load package
include("/Users/kevin/Jupyter/Hydraulics/Hydraulics_1.jl") # my PSV library
# using Hydraulics


thread = 1 warning: only found 9 / 10 columns around data row: 37. Filling remaining columns with `missing`


Main.Hydraulics

List a common utility table.

In [58]:
Hydraulics.npsList;

## Format of input data
Input data is required the following tables.
* fluidList: this tabulates fluid density and viscosity for a given fluid. Used for sizing and hydraulic calculations.

Sizing
* Put the sizing information in the table lineSizing. A spreadsheet is very convenient becauase it allows one to use descriptors for properties (such as fluid type, pipe material, pipe schedule).

Hydraulics
* Need a table (lines) for describing overall conditions for each line segment.
* also need a table (fittingList) for listing all of the piping components in each line segment.

A spreadsheet is very convenient for tabulating the input data. Dropdown menus are used to ensure inputs are correct (fluid type, NPS, pipe schedule). Then the hydraulic calculations are performed using the input data in the spreadsheet.



In [59]:
# these are the specific data
lineSizing = DataFrame(load("hydraulicDescription_2.xlsx", "sizing"))
lineHydraulics = DataFrame(load("hydraulicDescription_2.xlsx", "hydraulics"))
fluidList = DataFrame(load("hydraulicDescription_2.xlsx", "fluidList"))
fittingList = DataFrame(load("hydraulicDescription_2.xlsx", "fittingList"));

## Check input tables for inconsistancies

In [60]:
Hydraulics.checkLineList(lineHydraulics,fluidList)

,message,entry,Segment,NPS,SCHED,material,fluidName
,String,Int64,String,Float64,String,String,String


In [61]:
Hydraulics.pipeRoughness;

## Line Sizing

The spreadsheet hydraulicDescription_2.xls contains the data needed for line sizing.

In [62]:
myLineSize = Hydraulics.getLineSize(lineSizing, fluidList)

,Segment,mmDP100,mmErosion,mmNeeded,Schedule
,String,Float64,Float64,Float64,String
1,A,293.178,314.414,314.414,STD
2,B,66.0907,60.752,66.0907,STD
3,C,53.3495,21.1511,53.3495,STD
4,D,44.0575,27.306,44.0575,STD


In [73]:
chosenLineSizes = Hydraulics.getListLargerNPS(myLineSize)

,Segment,NPS,Schedule,IDmm
,String,Float64,String,Float64
1,A,14.0,STD,336.54
2,B,3.0,STD,77.9
3,C,2.5,STD,62.6
4,D,2.0,STD,52.5


In [74]:
Hydraulics.incrementLineSize(3,chosenLineSizes,1)
chosenLineSizes

,Segment,NPS,Schedule,IDmm
,String,Float64,String,Float64
1,A,14.0,STD,336.54
2,B,3.0,STD,77.9
3,C,3.0,STD,77.9
4,D,2.0,STD,52.5


In [75]:
Hydraulics.sizing2hydraulics(lineSizing, chosenLineSizes)

,Segment,subSegment,Description,LineTag,PnID,NPS,Schedule,Material
,String,String,String,String,String,Float64,String,String
1,A,,Test line,L-001,PID-007,14.0,STD,Commercial steel
2,B,,Testline2,L-002,PID-007,3.0,STD,Commercial steel
3,C,,Testline3,L-003,PID-007,3.0,STD,Commercial steel
4,D,,Testline3,L-004,PID-007,2.0,STD,Commercial steel


In [67]:
Hydraulics.sizing2fittingList(lineSizing, chosenLineSizes)

,Segment,fittingType,num_length_m,comment,revision
,String,String,Float64,String,String
1,A,PIPE,100.0,from line sizing,A
2,B,PIPE,100.0,from line sizing,A
3,C,PIPE,100.0,from line sizing,A
4,D,PIPE,100.0,from line sizing,A


## Hydraulic calculations

First, take a look at the input data.

In [76]:
fluidList

,fluidName,rho_kgm3,mu_mPas,Comment
,String,Float64,Float64,String?
1,water,1000.0,0.99,missing
2,propane,500.0,0.1,missing
3,Air_1,8.47359,0.015,700 kPaa - 15 C


In [69]:
lineHydraulics

,Segment,subSegment,Description,LineTag,PnID,NPS,Schedule,Material
,String,String,String,String,String,Float64,String,String
1,A,blank,one line,L-001,optional,10.0,STD,Commercial steel
2,B,blank,another line,L-002,optional,8.0,STD,Commercial steel


In [70]:
fittingList

,Segment,subSegment,fittingType,num_length_m,comment,revision
,String,String,String,Float64,String?,String?
1,A,blank,EL45-THD-STD,2.0,missing,missing
2,A,blank,PIPE,55.0,smurf,B
3,B,blank,PIPE,100.0,missing,missing


Get pressure drop for each segment.

In [71]:
myDP = Hydraulics.getSegmentDP(lineHydraulics,fittingList,fluidList)

,Segment,segmentK,DPkpa,inletP,outletP
,String,Float64,Float64,Float64,Float64
1,A,4.26643,0.915928,100.0,100.916
2,B,7.83933,2.09113,100.0,102.091
